# Babi End to End MemNN

In [1]:
%matplotlib inline

In [3]:
import importlib, utils_babi_memnn; importlib.reload(utils_babi_memnn)
from utils_babi_memnn import *

Using TensorFlow backend.
C:\Users\Gavrilov\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\Gavrilov\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
np.set_printoptions(4)
cfg = K.tf.ConfigProto(gpu_options={'allow_growth': True})
K.set_session(K.tf.Session(config=cfg))

A memory network is a network that can retain information; it can be trained on a structured story and will learn how to answer questions about said story.

This notebook contains an implementation of an end-to-end memory network trained on the Babi tasks dataset.

## Create datasets

Code from this section is mainly taken from the babi-memnn example in the keras repo.

* [Popular Science](http://www.popsci.com/facebook-ai)
* [Slate](http://www.slate.com/blogs/future_tense/2016/06/28/facebook_s_ai_researchers_are_making_bots_smarter_by_giving_them_memory.html)

The Babi dataset is a collection of tasks (or stories) that detail events in a particular format. At the end of each task is a question with a labelled answer.

This section shows how to construct the dataset from the raw data.

In [5]:
def tokenize(sent):
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]

This parser formats the story into a time-order labelled sequence of sentences, followed by the question and the labelled answer.

In [6]:
def parse_stories(lines):
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        if int(nid) == 1: story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = None
            substory = [[str(i)+":"]+x for i,x in enumerate(story) if x]
            data.append((substory, q, a))
            story.append('')
        else: story.append(tokenize(line))
    return data

Next we download and parse the data set.

In [8]:
path = get_file('babi-tasks-v1-2.tar.gz', 
                origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
tar = tarfile.open(path)

11747328/11745123 [==============================] - ETA: 3: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 56s - ETA: 45 - ETA: 34 - ETA: 33 - ETA: 27 - ETA: 26 - ETA: 22 - ETA: 20 - ETA: 18 - ETA: 15 - ETA: 16 - ETA: 13 - ETA: 12 - ETA: 11 - ETA: 10 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 8s 1us/step


### Different text files for various different tasks

#### goal is to be able to solve every one of the challanges with just a thousand examples

In [9]:
challenges = {
    # QA1 with 10,000 samples
    'single_supporting_fact_10k': 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt',
    # QA2 with 10,000 samples
    'two_supporting_facts_10k': 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt',
    'two_supporting_facts_1k': 'tasks_1-20_v1-2/en/qa2_two-supporting-facts_{}.txt',
}
challenge_type = 'single_supporting_fact_10k'
# challenge_type = 'two_supporting_facts_10k'
challenge = challenges[challenge_type]

#### we going to put that into bunch of different lists of stories along with their queries

In [10]:
def get_stories(f):
    data = parse_stories(f.readlines())
    return [(story, q, answer) for story, q, answer in data]

In [11]:
train_stories = get_stories(tar.extractfile(challenge.format('train')))
test_stories = get_stories(tar.extractfile(challenge.format('test')))

C:\Users\Gavrilov\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


Here we calculate upper bounds for things like words in sentence, sentences in a story, etc. for the corpus, which will be useful later.

In [12]:
stories = train_stories + test_stories

In [13]:
story_maxlen = max((len(s) for x, _, _ in stories for s in x))
story_maxsents = max((len(x) for x, _, _ in stories))
query_maxlen = max(len(x) for _, x, _ in stories)

In [14]:
def do_flatten(el): 
    return isinstance(el, collections.Iterable) and not isinstance(el, (str, bytes))
def flatten(l):
    for el in l:
        if do_flatten(el): yield from flatten(el)
        else: yield el

Create vocabulary of corpus and find size, including a padding element.

In [15]:
vocab = sorted(set(flatten(stories)))
vocab.insert(0, '<PAD>')
vocab_size = len(vocab)

#### stories statistics

In [16]:
story_maxsents, vocab_size, story_maxlen, query_maxlen, len(train_stories), len(test_stories)

(10, 32, 8, 4, 10000, 1000)

Now the dataset is in the correct format.

Each task in the dataset contains a list of tokenized sentences ordered in time, followed by a question about the story with a given answer.

In the example below, we go can backward through the sentences to find the answer to the question "Where is Daniel?" as sentence 12, the last sentence to mention Daniel.

This task structure is called a "one supporting fact" structure, which means that we only need to find one sentence in the story to answer our question.

In [19]:
test_stories[534] #example of babi task, i.e. 1-supporting-fact structure 
#each task os basically a story
#at the end of the story is a query ti which there is an answer

([['0:', 'Mary', 'moved', 'to', 'the', 'office', '.'],
  ['1:', 'John', 'moved', 'to', 'the', 'garden', '.'],
  ['3:', 'Sandra', 'moved', 'to', 'the', 'bedroom', '.'],
  ['4:', 'Sandra', 'went', 'back', 'to', 'the', 'office', '.'],
  ['6:', 'John', 'went', 'to', 'the', 'bedroom', '.'],
  ['7:', 'John', 'journeyed', 'to', 'the', 'garden', '.'],
  ['9:', 'Daniel', 'went', 'back', 'to', 'the', 'hallway', '.'],
  ['10:', 'John', 'journeyed', 'to', 'the', 'bedroom', '.'],
  ['12:', 'Daniel', 'journeyed', 'to', 'the', 'bathroom', '.'],
  ['13:', 'John', 'travelled', 'to', 'the', 'garden', '.']],
 ['Where', 'is', 'Daniel', '?'],
 'bathroom')

Create an index mapping for the vocabulary.

In [20]:
word_idx = dict((c, i) for i, c in enumerate(vocab))
#dictionary from words to integers 

Next we vectorize our dataset by mapping words to their indices. We enforce consistent dimension by padding vectors up to the upper bounds we calculated earlier with our pad element.

In [21]:
def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    X = []; Xq = []; Y = []
    for story, query, answer in data:
        x = [[word_idx[w] for w in s] for s in story]
        xq = [word_idx[w] for w in query]
        y = [word_idx[answer]]
        X.append(x); Xq.append(xq); Y.append(y)
    return ([pad_sequences(x, maxlen=story_maxlen) for x in X],
            pad_sequences(Xq, maxlen=query_maxlen), np.array(Y))
#to get this into Keras we need to turn it into a tensor in which everything is the same size, so we use pad_sequences

In [22]:
inputs_train, queries_train, answers_train = vectorize_stories(train_stories, 
     word_idx, story_maxlen, query_maxlen)
inputs_test, queries_test, answers_test = vectorize_stories(test_stories, 
     word_idx, story_maxlen, query_maxlen)

In [23]:
def stack_inputs(inputs):
    for i,it in enumerate(inputs):
        inputs[i] = np.concatenate([it, 
                           np.zeros((story_maxsents-it.shape[0],story_maxlen), 'int')])
    return np.stack(inputs)
inputs_train = stack_inputs(inputs_train)
inputs_test = stack_inputs(inputs_test)

In [24]:
inputs_train.shape, inputs_test.shape

((10000, 10, 8), (1000, 10, 8))

Our inputs for keras.

In [25]:
inps = [inputs_train, queries_train]
val_inps = [inputs_test, queries_test]

## Model

The approach to solving this task relies not only on word embeddings, but sentence embeddings.

The authors of the Babi paper constructed sentence embeddings by simply adding up the word embeddings; this might seem naive, but given the relatively small length of these sentences we can expect the sum to capture relevant information.

In [26]:
emb_dim = 20
parms = {'verbose': 2, 'callbacks': [TQDMNotebookCallback(leave_inner=False)]}

We use <tt>TimeDistributed</tt> here to apply the embedding to every element of the sequence, then the <tt>Lambda</tt> layer adds them up

In [27]:
#we have to turn sentence into embedding not just a word
#here we just adding embeddings up
def emb_sent_bow(inp):
    emb = TimeDistributed(Embedding(vocab_size, emb_dim))(inp) #taking each word and putting it through separate embedding, this is what Timedistributed doing 
    return Lambda(lambda x: K.sum(x, 2))(emb) #then we do lambda layer to add them up

The embedding works as desired; the raw input has 10 sentences of 8 words, and the output has 10 sentence embeddings of length 20.

In [28]:
inp_story = Input((story_maxsents, story_maxlen)) #creating input for stories
emb_story = emb_sent_bow(inp_story)
inp_story.shape, emb_story.shape
#we not going to use word2vec or anything because we don't need the complexity of that 

(TensorShape([Dimension(None), Dimension(10), Dimension(8)]),
 TensorShape([Dimension(None), Dimension(10), Dimension(20)]))

We do the same for the queries, omitting the <tt>TimeDistributed</tt> since there is only one query. We use <tt>Reshape</tt> to match the rank of the input.

In [29]:
inp_q = Input((query_maxlen,))
emb_q = Embedding(vocab_size, emb_dim)(inp_q)
emb_q = Lambda(lambda x: K.sum(x, 1))(emb_q)
emb_q = Reshape((1, emb_dim))(emb_q)
inp_q.shape, emb_q.shape

(TensorShape([Dimension(None), Dimension(4)]),
 TensorShape([Dimension(None), Dimension(1), Dimension(20)]))

The actual memory network is incredibly simple.

* For each story, we take the dot product of every sentence embedding with that story's query embedding. This gives us a list of numbers proportional to how similar each sentence is with the query.
* We pass this vector of dot products through a softmax function to return a list of scalars that sum to one and tell us how similar the query is to each sentence.

In [30]:
#basically this is our end-to-end memory network
x = merge([emb_story, emb_q], mode='dot', dot_axes=2) #we've got the story times the query, dot product 
x = Reshape((story_maxsents,))(x) #reshape to get rid of the unit axes, because softmax works from the last dimension, and reshape again
x = Activation('softmax')(x) #
match = Reshape((story_maxsents,1))(x) #reshape again to put unit axes back on again #dot product followed by soft max gives us the weight
match.shape

C:\Users\Gavrilov\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  """Entry point for launching an IPython kernel.
C:\Users\Gavrilov\Anaconda3\lib\site-packages\keras\legacy\layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


TensorShape([Dimension(None), Dimension(10), Dimension(1)])

* Next, we construct a second, separate, embedding function for the sentences
* We then take the weighted average of these embeddings, using the softmax outputs as weights
* Finally, we pass this weighted average though a dense layer and classify it w/ a softmax into one of the words in the vocabulary

In [31]:
emb_c = emb_sent_bow(inp_story) #we need to take each weigh and multiply it by the second set of embeddings
x = merge([match, emb_c], mode='dot', dot_axes=1) #in order to do this we just use dot product again
#because of the fact that we've got a unit axis there, this is just doing a very simple weighted average
response = Reshape((emb_dim,))(x) #reshape again to get rid of the unit axes so we can stick it through a dense layer with a soft max
res = Dense(vocab_size, activation='softmax')(response)

C:\Users\Gavrilov\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
C:\Users\Gavrilov\Anaconda3\lib\site-packages\keras\legacy\layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [32]:
answer = Model([inp_story, inp_q], res)

In [33]:
answer.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

And it works extremely well

In [34]:
K.set_value(answer.optimizer.lr, 1e-2)
hist=answer.fit(inps, answers_train, **parms, nb_epoch=4, batch_size=32,
           validation_data=(val_inps, answers_test))

C:\Users\Gavrilov\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 10000 samples, validate on 1000 samples


A Jupyter Widget

A Jupyter Widget

Epoch 1/4
 - 5s - loss: 0.7780 - acc: 0.6940 - val_loss: 2.1854e-04 - val_acc: 1.0000


...

Epoch 2/4
 - 4s - loss: 0.0084 - acc: 0.9981 - val_loss: 0.0013 - val_acc: 1.0000


A Jupyter Widget

Epoch 3/4
 - 3s - loss: 0.0011 - acc: 0.9995 - val_loss: 3.2895e-06 - val_acc: 1.0000


A Jupyter Widget

Epoch 4/4
 - 3s - loss: 0.0062 - acc: 0.9991 - val_loss: 0.0931 - val_acc: 0.9830



## Test

We can look inside our model to see how it's weighting the sentence embeddings.

In [35]:
f = Model([inp_story, inp_q], match) #another model which is already trained to return value of weights

In [36]:
qnum=6

In [37]:
#we are looking for weights for a particular story
l_st = len(train_stories[qnum][0])+1
train_stories[qnum]

([['0:', 'Sandra', 'travelled', 'to', 'the', 'office', '.'],
  ['1:', 'Sandra', 'went', 'to', 'the', 'bathroom', '.'],
  ['3:', 'Mary', 'went', 'to', 'the', 'bedroom', '.'],
  ['4:', 'Daniel', 'moved', 'to', 'the', 'hallway', '.']],
 ['Where', 'is', 'Sandra', '?'],
 'bathroom')

Sure enough, for the question "Where is Sandra?", the largest weight is the last sentence with the name Sandra, sentence 1 with 0.98.

The second highest is of course the first sentence, which also mentions Sandra. But the model has learned that the last occurring sentence is what is important; this is why we added the counter at the beginning of each sentence.

In [115]:
np.squeeze(answer.predict([inputs_train[qnum:qnum+1], queries_train[qnum:qnum+1]]))[:l_st]

array([  4.5973e-13,   5.5948e-13,   3.2305e-13,   4.1206e-12,   3.5531e-13], dtype=float32)

In [38]:
np.squeeze(f.predict([inputs_train[qnum:qnum+1], queries_train[qnum:qnum+1]]))[:l_st] #predicting model f, rather than answer.predict 
#for particular story we have following weights
#we can look inside and find out what sentence is it using to answer this question

array([  3.0545e-03,   9.9692e-01,   1.5097e-07,   2.7239e-05,   4.0694e-14], dtype=float32)

In [39]:
answers_train[qnum:qnum+10,0]

array([19, 19, 27, 22, 19, 20, 19, 19, 20, 20])

In [40]:
np.argmax(answer.predict([inputs_train[qnum:qnum+10], queries_train[qnum:qnum+10]]), 1)

array([19, 19, 27, 22, 19, 20, 19, 19, 20, 20], dtype=int64)

In [41]:
answer.predict([inputs_train[qnum:qnum+1], queries_train[qnum:qnum+1]])

array([[  4.5973e-13,   5.5948e-13,   3.2305e-13,   4.1206e-12,
          3.5531e-13,   2.1627e-12,   7.4939e-13,   5.1864e-13,
          3.3405e-13,   6.9721e-13,   2.8543e-13,   1.7466e-13,
          1.0991e-12,   1.5451e-13,   1.8389e-13,   3.2007e-13,
          8.5982e-13,   2.6927e-13,   2.5998e-13,   1.0000e+00,
          1.8841e-09,   4.7756e-09,   1.0074e-10,   3.5300e-13,
          1.9326e-13,   2.5940e-11,   7.9140e-13,   3.0629e-09,
          1.1448e-13,   6.0000e-13,   8.0712e-13,   4.2092e-13]], dtype=float32)

In [42]:
vocab[19]

'bathroom'

## Multi hop

## Next, let's look at an example of a 2-supporting fact story

In [111]:
test_stories[534] #same example as previously

([['0:', 'Mary', 'moved', 'to', 'the', 'office', '.'],
  ['1:', 'John', 'moved', 'to', 'the', 'garden', '.'],
  ['3:', 'Sandra', 'moved', 'to', 'the', 'bedroom', '.'],
  ['4:', 'Sandra', 'went', 'back', 'to', 'the', 'office', '.'],
  ['6:', 'John', 'went', 'to', 'the', 'bedroom', '.'],
  ['7:', 'John', 'journeyed', 'to', 'the', 'garden', '.'],
  ['9:', 'Daniel', 'went', 'back', 'to', 'the', 'hallway', '.'],
  ['10:', 'John', 'journeyed', 'to', 'the', 'bedroom', '.'],
  ['12:', 'Daniel', 'journeyed', 'to', 'the', 'bathroom', '.'],
  ['13:', 'John', 'travelled', 'to', 'the', 'garden', '.']],
 ['Where', 'is', 'Daniel', '?'],
 'bathroom')

We can see that the question "Where is the milk?" requires to supporting facts to answer, "Daniel traveled to the hallway" and "Daniel left the milk there".

In [116]:
inputs_train.shape, inputs_test.shape

((10000, 10, 8), (1000, 10, 8))

The approach is basically the same; we add more embedding dimensions to account for the increased task complexity.

In [117]:
parms = {'verbose': 2, 'callbacks': [TQDMNotebookCallback(leave_inner=False)]}
emb_dim = 30

In [118]:
def emb_sent_bow(inp):
    emb_op = TimeDistributed(Embedding(vocab_size, emb_dim))
    emb = emb_op(inp)
    emb = Lambda(lambda x: K.sum(x, 2))(emb)
#     return Elemwise(0, False)(emb), emb_op
    return emb, emb_op

In [119]:
inp_story = Input((story_maxsents, story_maxlen))
inp_q = Input((query_maxlen,))

In [120]:
emb_story, emb_story_op = emb_sent_bow(inp_story)

In [121]:
emb_q = emb_story_op.layer(inp_q)
emb_q = Lambda(lambda x: K.sum(x, 1))(emb_q)

In [122]:
h = Dense(emb_dim)

The main difference is that we are going to do the same process twice. Here we've defined a "hop" as the operation that returns the weighted average of the input sentence embeddings.

In [123]:
def one_hop(u, A):
    C, _ = emb_sent_bow(inp_story)
    x = Reshape((1, emb_dim))(u)
    x = merge([A, x], mode='dot', dot_axes=2)
    x = Reshape((story_maxsents,))(x)
    x = Activation('softmax')(x)
    match = Reshape((story_maxsents,1))(x)

    x = merge([match, C], mode='dot', dot_axes=1)
    x = Reshape((emb_dim,))(x)
    x = h(x)
    x = merge([x, emb_q], 'sum')
    return x, C

We do one hop, and repeat the process using the resulting weighted sentence average as the new weights.

This works because the first hop allows us to find the first fact relevant to the query, and then we can use that fact to find the next fact that answers the question. In our example, our model would first find the last sentence to mention "milk", and then use the information in that fact to know that it next has to find the last occurrence of "Daniel".

This is facilitated by generating a new embedding function for the input story each time we hop. This means that the first embedding is learning things that help us find the first fact from the query, and the second is helping us find the second fact from the first.

This approach can be extended to n-supporting factor problems by doing n hops.

In [124]:
response, emb_story = one_hop(emb_q, emb_story)
response, emb_story = one_hop(response, emb_story)
# response, emb_story = one_hop(response, emb_story)

C:\Users\Gavrilov\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  after removing the cwd from sys.path.
C:\Users\Gavrilov\Anaconda3\lib\site-packages\keras\legacy\layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
C:\Users\Gavrilov\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  if __name__ == '__main__':
C:\Users\Gavrilov\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concaten

In [125]:
res = Dense(vocab_size, activation='softmax')(response)

In [126]:
answer = Model([inp_story, inp_q], res)
answer.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

Fitting this model can be tricky.

In [127]:
K.set_value(answer.optimizer.lr, 5e-3)
hist=answer.fit(inps, answers_train, **parms, nb_epoch=8, batch_size=32,
           validation_data=(val_inps, answers_test))

C:\Users\Gavrilov\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 10000 samples, validate on 1000 samples


A Jupyter Widget

A Jupyter Widget

Epoch 1/8
 - 8s - loss: 0.6305 - acc: 0.7749 - val_loss: 0.0046 - val_acc: 0.9990


...

Epoch 6/8
 - 6s - loss: 0.0059 - acc: 0.9993 - val_loss: 3.3560e-06 - val_acc: 1.0000


A Jupyter Widget

Epoch 7/8
 - 6s - loss: 0.0122 - acc: 0.9988 - val_loss: 1.8073e-05 - val_acc: 1.0000


A Jupyter Widget

Epoch 8/8
 - 6s - loss: 0.0115 - acc: 0.9986 - val_loss: 3.7416e-06 - val_acc: 1.0000



In [129]:
np.squeeze(answer.predict([inputs_train[qnum:qnum+1], queries_train[qnum:qnum+1]]))[:l_st]

array([  2.2865e-11,   4.3573e-12,   3.3579e-12,   1.1121e-12,   3.7014e-12], dtype=float32)

In [130]:
l_st = len(train_stories[qnum][0])+1
train_stories[qnum]

([['0:', 'Sandra', 'travelled', 'to', 'the', 'office', '.'],
  ['1:', 'Sandra', 'went', 'to', 'the', 'bathroom', '.'],
  ['3:', 'Mary', 'went', 'to', 'the', 'bedroom', '.'],
  ['4:', 'Daniel', 'moved', 'to', 'the', 'hallway', '.']],
 ['Where', 'is', 'Sandra', '?'],
 'bathroom')

In [128]:
np.array(hist.history['val_acc'])

array([ 0.999,  0.999,  1.   ,  0.991,  1.   ,  1.   ,  1.   ,  1.   ])